In [1]:
import os

import keras
import numpy as np
import sklearn.metrics
import tensorflow as tf

from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import average, AveragePooling2D, concatenate, Conv2D, Conv3D, Dense, Flatten, Input, Reshape
from keras.models import Model, Sequential
from keras.optimizers import Adam

PATCH_HEIGHT = 28
PATCH_WIDTH = 28

data_dir = 'data'
if not os.path.exists('checkpoints'):
    os.mkdir('checkpoints')
if not os.path.exists('logs'):
    os.mkdir('logs')

Using TensorFlow backend.


In [2]:
ct_train = np.load(os.path.join(data_dir, 'ct_train.npy'))
pet_train = np.load(os.path.join(data_dir, 'pet_train.npy'))
y_train = np.load(os.path.join(data_dir, 'y_train.npy'))
ct_test = np.load(os.path.join(data_dir, 'ct_test.npy'))
pet_test = np.load(os.path.join(data_dir, 'pet_test.npy'))
y_test = np.load(os.path.join(data_dir, 'y_test.npy'))

In [3]:
def confusion_matrix(y_true, y_pred):
    y_true_targets = np.argmax(y_true, axis=1)
    y_pred_targets = np.argmax(y_pred, axis=1)
    return sklearn.metrics.confusion_matrix(y_true_targets, y_pred_targets)

def accuracy(y_true, y_pred):
    y_true_targets = np.argmax(y_true, axis=1)
    y_pred_targets = np.argmax(y_pred, axis=1)
    return sklearn.metrics.accuracy_score(y_true_targets, y_pred_targets)

In [4]:
def train_model(model, name, batch_size=32, epochs=3, patience=0, data=None, save=False):
    print('Train...')

    best_model_path = os.path.join('checkpoints', f'best_model_{name}.h5')
    log_dir = os.path.join('logs', f'{name}')

    if not os.path.exists(log_dir):
        os.mkdir(log_dir)

    callbacks = [EarlyStopping(monitor='val_acc', patience=patience)]
    
    if save:
        callbacks.append(ModelCheckpoint(best_model_path, monitor='val_acc', save_best_only=True, save_weights_only=True))
        callbacks.append(TensorBoard(log_dir=log_dir, histogram_freq=1, batch_size=batch_size, write_graph=False, write_grads=True, write_images=True))
    
    if data == 'ct':
        x_train = ct_train
        x_test = ct_test
    elif data == 'pet':
        x_train = pet_train
        x_test = pet_test
    else:
        x_train = [ct_train, pet_train]
        x_test = [ct_test, pet_test]

    model.fit(x_train,
              y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              verbose=1,
              shuffle=True,
              callbacks=callbacks)

def train_n_sessions(model_fn, name, n, data=None, **kwargs):
    c_matrices = []
    accuracies = []
    
    if data == 'ct':
        x_test = ct_test
    elif data == 'pet':
        x_test = pet_test
    else:
        x_test = [ct_test, pet_test]
    
    for i in range(n):
        print(f'Round {i + 1} out of {n}')
        print('-' * 101)
        model = model_fn()
        train_model(model, name, data=data, **kwargs)
        y_preds = model.predict(x_test)
        c_matrix = confusion_matrix(y_test, y_preds)
        accuracy_score = accuracy(y_test, y_preds)
        c_matrices.append(c_matrix)
        accuracies.append(accuracy_score)
        
        print(c_matrix)
        print(accuracy_score)
        print('\n\n')
    
    return c_matrices, accuracies

# Type 1: Feature-Level Fusion

In [5]:
def get_type_1_model(summary=False):
    K.clear_session()

    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))

    x = concatenate([ct_input, pet_input])
    x = Reshape((PATCH_HEIGHT, PATCH_WIDTH, 2, 1))(x)
    x = Conv3D(16, (2, 2, 2), activation='relu')(x)
    x = Reshape((27, 27, 16))(x)
    x = Conv2D(36, (2, 2), activation='relu')(x)
    x = Conv2D(64, (2, 2), activation='relu')(x)
    x = Conv2D(144, (2, 2), activation='relu')(x)
    x = AveragePooling2D((23, 23))(x)
    x = Flatten()(x)
    x = Dense(864, activation='relu')(x)
    x = Dense(288, activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=[ct_input, pet_input], outputs=output)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()

    return model

In [6]:
c_matrices_1, accuracies_1 = train_n_sessions(get_type_1_model, 'type_I', 10)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Train on 41338 samples, validate on 4212 samples
Epoch 1/3
41338/41338 [==============================] - 27s - loss: 0.1207 - acc: 0.9579 - val_loss: 0.2426 - val_acc: 0.9179
Epoch 2/3
41338/41338 [==============================] - 23s - loss: 0.0787 - acc: 0.9739 - val_loss: 0.4265 - val_acc: 0.9145
[[1990  116]
 [ 244 1862]]
0.91452991453



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Train on 41338 samples, validate on 4212 samples
Epoch 1/3
41338/41338 [==============================] - 23s - loss: 0.1205 - acc: 0.9585 - val_loss: 0.1776 - val_acc: 0.9421
Epoch 2/3
41338/41338 [==============================] - 23s - loss: 0.0834 - acc: 0.9708 - val_loss: 0.1992 - val_acc: 0.9328
[[1940  166]
 [ 117 1989]]
0.932811016144



Round 3 out of 10
-----------------------------

In [7]:
print(c_matrices_1)
print(accuracies_1)

[array([[1990,  116],
       [ 244, 1862]]), array([[1940,  166],
       [ 117, 1989]]), array([[1991,  115],
       [ 260, 1846]]), array([[1946,  160],
       [ 260, 1846]]), array([[1983,  123],
       [ 252, 1854]]), array([[1948,  158],
       [ 213, 1893]]), array([[1942,  164],
       [  99, 2007]]), array([[1996,  110],
       [ 134, 1972]]), array([[1967,  139],
       [ 253, 1853]]), array([[1932,  174],
       [ 121, 1985]])]
[0.9145299145299145, 0.93281101614434947, 0.91096866096866091, 0.90028490028490027, 0.91096866096866091, 0.91191832858499522, 0.93755935422602088, 0.94207027540360877, 0.90693257359924029, 0.92996201329534667]


# Type 2: Classifier-Level Fusion

In [8]:
def get_type_2_model(summary=False):
    K.clear_session()

    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))

    ct_model = Conv2D(16, (2, 2), activation='relu')(ct_input)
    ct_model = Conv2D(36, (2, 2), activation='relu')(ct_model)
    ct_model = Conv2D(64, (2, 2), activation='relu')(ct_model)
    ct_model = Conv2D(144, (2, 2), activation='relu')(ct_model)
    ct_model = AveragePooling2D((23, 23))(ct_model)
    ct_model = Flatten()(ct_model)

    pet_model = Conv2D(16, (2, 2), activation='relu')(pet_input)
    pet_model = Conv2D(36, (2, 2), activation='relu')(pet_model)
    pet_model = Conv2D(64, (2, 2), activation='relu')(pet_model)
    pet_model = Conv2D(144, (2, 2), activation='relu')(pet_model)
    pet_model = AveragePooling2D((23, 23))(pet_model)
    pet_model = Flatten()(pet_model)

    x = concatenate([ct_model, pet_model])
    x = Dense(864, activation='relu')(x)
    x = Dense(288, activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=[ct_input, pet_input], outputs=output)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()
    
    return model

In [9]:
c_matrices_2, accuracies_2 = train_n_sessions(get_type_2_model, 'type_II', 10)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Train on 41338 samples, validate on 4212 samples
Epoch 1/3
41338/41338 [==============================] - 41s - loss: 0.1104 - acc: 0.9609 - val_loss: 0.2222 - val_acc: 0.9378
Epoch 2/3
41338/41338 [==============================] - 41s - loss: 0.0703 - acc: 0.9775 - val_loss: 0.2446 - val_acc: 0.9392
Epoch 3/3
41338/41338 [==============================] - 41s - loss: 0.0555 - acc: 0.9823 - val_loss: 0.2063 - val_acc: 0.9345
[[2010   96]
 [ 180 1926]]
0.934472934473



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Train on 41338 samples, validate on 4212 samples
Epoch 1/3
41338/41338 [==============================] - 41s - loss: 0.1126 - acc: 0.9615 - val_loss: 0.1896 - val_acc: 0.9376
Epoch 2/3
41338/41338 [==============================] - 41s - loss: 0.0763 - acc: 0.9751 -

In [10]:
print(c_matrices_2)
print(accuracies_2)

[array([[2010,   96],
       [ 180, 1926]]), array([[2000,  106],
       [ 159, 1947]]), array([[1966,  140],
       [  81, 2025]]), array([[1996,  110],
       [ 244, 1862]]), array([[2037,   69],
       [ 268, 1838]]), array([[1927,  179],
       [ 176, 1930]]), array([[1975,  131],
       [ 131, 1975]]), array([[1956,  150],
       [ 120, 1986]]), array([[1985,  121],
       [ 130, 1976]]), array([[1871,  235],
       [  41, 2065]])]
[0.93447293447293445, 0.93708452041785373, 0.94753086419753085, 0.91595441595441596, 0.91999050332383669, 0.91571699905033244, 0.93779677113010451, 0.9358974358974359, 0.94040835707502379, 0.93447293447293445]


# Type 3: Decision-Level Fusion

In [5]:
def get_type_3_model(summary=False):
    K.clear_session()

    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))

    ct_model = Conv2D(16, (2, 2), activation='relu')(ct_input)
    ct_model = Conv2D(36, (2, 2), activation='relu')(ct_model)
    ct_model = Conv2D(64, (2, 2), activation='relu')(ct_model)
    ct_model = Conv2D(144, (2, 2), activation='relu')(ct_model)
    ct_model = AveragePooling2D((23, 23))(ct_model)
    ct_model = Flatten()(ct_model)
    ct_model = Dense(864, activation='relu')(ct_model)
    ct_model = Dense(288, activation='relu')(ct_model)
    ct_model = Dense(2, activation='softmax')(ct_model)

    pet_model = Conv2D(16, (2, 2), activation='relu')(pet_input)
    pet_model = Conv2D(36, (2, 2), activation='relu')(pet_model)
    pet_model = Conv2D(64, (2, 2), activation='relu')(pet_model)
    pet_model = Conv2D(144, (2, 2), activation='relu')(pet_model)
    pet_model = AveragePooling2D((23, 23))(pet_model)
    pet_model = Flatten()(pet_model)
    pet_model = Dense(864, activation='relu')(pet_model)
    pet_model = Dense(288, activation='relu')(pet_model)
    pet_model = Dense(2, activation='softmax')(pet_model)

    predictions = average([ct_model, pet_model])

    model = Model(inputs=[ct_input, pet_input], outputs=predictions)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()
    
    return model

In [8]:
c_matrices_3, accuracies_3 = train_n_sessions(get_type_3_model, 'type_III', 10)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Train on 41338 samples, validate on 4212 samples
Epoch 1/3
41338/41338 [==============================] - 43s - loss: 0.1441 - acc: 0.9474 - val_loss: 0.2587 - val_acc: 0.8443
Epoch 2/3
41338/41338 [==============================] - 43s - loss: 0.1112 - acc: 0.9596 - val_loss: 0.2688 - val_acc: 0.8504
Epoch 3/3
41338/41338 [==============================] - 43s - loss: 0.0996 - acc: 0.9611 - val_loss: 0.2665 - val_acc: 0.8720
[[1995  111]
 [ 428 1678]]
0.872032288699



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Train on 41338 samples, validate on 4212 samples
Epoch 1/3
41338/41338 [==============================] - 43s - loss: 0.1409 - acc: 0.9567 - val_loss: 0.2876 - val_acc: 0.8561
Epoch 2/3
41338/41338 [==============================] - 43s - loss: 0.0969 - acc: 0.9697 -

In [9]:
print(c_matrices_3)
print(accuracies_3)

[array([[1995,  111],
       [ 428, 1678]]), array([[2003,  103],
       [ 385, 1721]]), array([[1996,  110],
       [ 421, 1685]]), array([[2011,   95],
       [ 518, 1588]]), array([[1975,  131],
       [ 454, 1652]]), array([[1929,  177],
       [ 231, 1875]]), array([[2011,   95],
       [ 466, 1640]]), array([[1965,  141],
       [ 319, 1787]]), array([[1987,  119],
       [ 204, 1902]]), array([[2039,   67],
       [ 566, 1540]])]
[0.87203228869895533, 0.88414055080721743, 0.87393162393162394, 0.85446343779677114, 0.86111111111111116, 0.90313390313390318, 0.86680911680911676, 0.89078822412155745, 0.92331433998100665, 0.84971509971509973]


# Baseline: Single-Modality CNNs

In [17]:
def get_single_modality_model(summary=False):
    print('Build model...')

    K.clear_session()
    
    model = Sequential()
    model.add(Conv2D(16, (2, 2), activation='relu', input_shape=(PATCH_HEIGHT, PATCH_WIDTH, 1)))
    model.add(Conv2D(36, (2, 2), activation='relu'))
    model.add(Conv2D(64, (2, 2), activation='relu'))
    model.add(Conv2D(144, (2, 2), activation='relu'))
    model.add(AveragePooling2D((23, 23)))
    model.add(Flatten())
    model.add(Dense(864, activation='relu'))
    model.add(Dense(288, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()

    print('Model built.')
    
    return model

In [18]:
c_matrices_ct, accuracies_ct = train_n_sessions(get_single_modality_model, 'ct', 10, data='ct')

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Build model...
Model built.
Train...
Train on 41338 samples, validate on 4212 samples
Epoch 1/3
41338/41338 [==============================] - 23s - loss: 0.2027 - acc: 0.9230 - val_loss: 1.1299 - val_acc: 0.7840
Epoch 2/3
41338/41338 [==============================] - 23s - loss: 0.1443 - acc: 0.9497 - val_loss: 0.5484 - val_acc: 0.7828
[[1649  457]
 [ 458 1648]]
0.782763532764



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Build model...
Model built.
Train...
Train on 41338 samples, validate on 4212 samples
Epoch 1/3
41338/41338 [==============================] - 23s - loss: 0.2318 - acc: 0.9098 - val_loss: 0.4587 - val_acc: 0.7980
Epoch 2/3
41338/41338 [==============================] - 23s - loss: 0.1595 - acc: 0.9424 - val_loss: 1.2321 - val_acc: 0.8018
Epoch 3/3
41338/41338 [============

In [19]:
print(c_matrices_ct)
print(accuracies_ct)

[array([[1649,  457],
       [ 458, 1648]]), array([[1881,  225],
       [ 713, 1393]]), array([[1900,  206],
       [ 720, 1386]]), array([[1842,  264],
       [ 623, 1483]]), array([[1880,  226],
       [ 852, 1254]]), array([[1921,  185],
       [ 746, 1360]]), array([[1862,  244],
       [ 714, 1392]]), array([[1877,  229],
       [ 653, 1453]]), array([[1833,  273],
       [ 614, 1492]]), array([[1904,  202],
       [ 795, 1311]])]
[0.78276353276353272, 0.77730294396961064, 0.78015194681861344, 0.78941120607787274, 0.74406457739791076, 0.77896486229819561, 0.77255460588793923, 0.79059829059829057, 0.78941120607787274, 0.76329534662867993]


In [20]:
c_matrices_pet, accuracies_pet = train_n_sessions(get_single_modality_model, 'pet', 10, data='pet')

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Build model...
Model built.
Train...
Train on 41338 samples, validate on 4212 samples
Epoch 1/3
41338/41338 [==============================] - 23s - loss: 0.1257 - acc: 0.9572 - val_loss: 0.1804 - val_acc: 0.9323
Epoch 2/3
41338/41338 [==============================] - 23s - loss: 0.0900 - acc: 0.9686 - val_loss: 0.3117 - val_acc: 0.9110
[[2018   88]
 [ 287 1819]]
0.910968660969



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Build model...
Model built.
Train...
Train on 41338 samples, validate on 4212 samples
Epoch 1/3
41338/41338 [==============================] - 24s - loss: 0.1182 - acc: 0.9585 - val_loss: 0.2166 - val_acc: 0.9174
Epoch 2/3
41338/41338 [==============================] - 23s - loss: 0.0899 - acc: 0.9692 - val_loss: 0.1786 - val_acc: 0.9366
Epoch 3/3
41338/41338 [============

In [21]:
print(c_matrices_pet)
print(accuracies_pet)

[array([[2018,   88],
       [ 287, 1819]]), array([[2000,  106],
       [ 362, 1744]]), array([[1986,  120],
       [ 247, 1859]]), array([[1962,  144],
       [ 152, 1954]]), array([[1982,  124],
       [ 185, 1921]]), array([[2041,   65],
       [ 442, 1664]]), array([[1978,  128],
       [  65, 2041]]), array([[2010,   96],
       [ 204, 1902]]), array([[2019,   87],
       [ 241, 1865]]), array([[1990,  116],
       [ 269, 1837]])]
[0.91096866096866091, 0.88888888888888884, 0.91286799620132952, 0.92972459639126304, 0.9266381766381766, 0.87962962962962965, 0.95417853751187087, 0.92877492877492873, 0.92212725546058882, 0.90859449192782527]


In [ ]:
def get_two_path_cascade(summary=False):
    K.clear_session()

    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    x = concatenate([ct_input, pet_input])
    x = Reshape((PATCH_HEIGHT, PATCH_WIDTH, 2, 1))(x)
    
    conv1_local = Conv2D(64, (7, 7), activation='relu')(x)
    pool1_local = AveragePooling2D((23, 23))(conv1_local)
    
    conv2_local = Conv2D(64, (3, 3), activation='relu')(pool1_local)
    pool2_local = AveragePooling2D((23, 23))(conv2_local)

    conv1_global= Conv2D(160,(13,13), activation= 'relu')(x)
    
    combine = merge([pool2_local,conv1_global], mode= 'concat', concat_axis=1)
    conv1_combine = Conv2D(5, (21,21), activation='relu')(combine)
    output = Flatten()(conv1_combine)
    
    output = Dense(2, activation='softmax')(output)

    model = Model(inputs=[ct_input, pet_input], outputs=output)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()
    return model


In [ ]:
c_matrices_two, accuracies_two = train_n_sessions(get_two_path_cascade, 'Casc-CNN-two', 10)

In [ ]:
def get_local_path_cascade(summary=False):
    K.clear_session()
    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    x = concatenate([ct_input, pet_input])
    x = Reshape((PATCH_HEIGHT, PATCH_WIDTH, 2, 1))(x)
    
    conv1_local = Conv2D(64, (7, 7), activation='relu')(x)
    pool1_local = AveragePooling2D((23, 23))(conv1_local)
    
    conv2_local = Conv2D(64, (3, 3), activation='relu')(pool1_local)
    pool2_local = AveragePooling2D((23,23))(conv2_local)
    
    conv1_combine = Conv2D(5, (21,21), activation= 'relu')(pool2_local)
    
    output = Flatten()(conv1_combine)
    output = Dense(2, activation= 'softmax')(output)
    
    model = Model(inputs=[ct_input, pet_input], outputs=output)
    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    if summary:
        model.summary()
        
    return model
    
    

In [ ]:
c_matrices_local, accuracies_local = train_n_sessions(get_local_path_cascade, 'Casc-CNN-local', 10)

In [ ]:
def get_global_path_cascade(summary=False):
    K.clear_session()
    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    x = concatenate([ct_input, pet_input])
    x = Reshape((PATCH_HEIGHT, PATCH_WIDTH, 2, 1))(x)
    
    conv1_local = Conv2D(160, (13, 13), activation='relu')(x)
    conv1_combine = Conv2D(5, (21,21) ,activation= 'relu')(conv1_local)
    
    output = Flatten()(conv1_combine)
    output = Dense(2, activation= 'softmax')(output)
    
    model = Model(inputs=[ct_input, pet_input], outputs=output)
    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    if summary:
        model.summary()
        
    return model

In [ ]:
c_matrices_global, accuracies_global = train_n_sessions(get_local_path_cascade, 'Casc-CNN-local', 10)